In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F

import torch.optim as optim
from torch.utils.data import DataLoader

import torchvision
import torchvision.transforms as transforms

from matplotlib.pyplot import imshow
%matplotlib inline

device = 'cuda' if torch.cuda.is_available() else 'cpu'

torch.manual_seed(777)
if device =='cuda':
    torch.cuda.manual_seed_all(777)

In [2]:
device

'cuda'

In [3]:
trans = transforms.Compose([
    transforms.Resize((400, 400)),
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])

train_data = torchvision.datasets.ImageFolder(root = 'Training', transform = trans)

In [4]:
data_loader = DataLoader(dataset = train_data, batch_size = 5, shuffle = True)

In [5]:
class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        self.layer1 = nn.Sequential(
            nn.Conv2d(3, 16, 5),
            nn.ReLU(),
            nn.MaxPool2d(2)
        )
        self.layer2 = nn.Sequential(
            nn.Conv2d(16, 32, 5),
            nn.ReLU(),
            nn.MaxPool2d(2)
        )
        self.layer3 = nn.Sequential(
            nn.Conv2d(32, 64, 5),
            nn.ReLU(),
            nn.MaxPool2d(2)
        )
        self.layer4 = nn.Sequential(
            nn.Conv2d(64, 128, 5),
            nn.ReLU(),
            nn.MaxPool2d(2)
        )
        self.layer5 = nn.Sequential(
            nn.Linear(128 * 21 * 21, 120),
            nn.ReLU(),
            nn.Linear(120, 2)
        )
    
    def forward(self, x):
        out = self.layer1(x)
        #print(out.shape)
        out = self.layer2(out)
        #print(out.shape)
        out = self.layer3(out)
        #print(out.shape)
        out = self.layer4(out)
        #print(out.shape)
        out = out.view(out.shape[0], -1)
        #print(out.shape)
        out = self.layer5(out)
        return out

In [6]:
#net = CNN().to(device)
#test_input = (torch.Tensor(3,3,400,400)).to(device)
#test_out = net(test_input)

In [7]:
net = CNN().to(device)
optimizer = optim.Adam(net.parameters(), lr = 0.0001)
loss_func = nn.CrossEntropyLoss().to(device)

In [8]:
total_batch = len(data_loader)

epochs = 100
for epoch in range(epochs):
    avg_cost = 0.0
    for num, data in enumerate(data_loader):
        imgs, labels = data
        imgs = imgs.to(device)
        labels = labels.to(device)
        optimizer.zero_grad()
        out = net(imgs)
        loss = loss_func(out, labels)
        loss.backward()
        optimizer.step()
        
        avg_cost += loss / total_batch
    
    if (epoch + 1) % 10 == 0:
        print('[Epoch:{}] cost = {}'.format(epoch + 1, avg_cost))
print('Learning Finished!')

[Epoch:10] cost = 0.39119911193847656
[Epoch:20] cost = 0.2548705041408539
[Epoch:30] cost = 0.17475862801074982
[Epoch:40] cost = 0.10319788008928299
[Epoch:50] cost = 0.056345753371715546
[Epoch:60] cost = 0.0377076230943203
[Epoch:70] cost = 0.024186082184314728
[Epoch:80] cost = 0.008995898999273777
[Epoch:90] cost = 0.003540225327014923
[Epoch:100] cost = 0.0021871135104447603
Learning Finished!


In [11]:
test_data = torchvision.datasets.ImageFolder(root = 'Test', transform = trans)
test_set = DataLoader(dataset = test_data, batch_size = 10, shuffle = True)

In [12]:
TotalAccuracy = 0

with torch.no_grad():
    for num, data in enumerate(test_set):
        imgs, label = data
        imgs = imgs.to(device)
        label = label.to(device)
        
        prediction = net(imgs)
        
        correct_prediction = torch.argmax(prediction, 1) == label
        
        accuracy = correct_prediction.float().mean()
        TotalAccuracy = TotalAccuracy + accuracy
    TotalAccuracy = TotalAccuracy / num
    print("-"*50)
    print("")
    print('Accuracy : ', TotalAccuracy.item())
    #print('F1 Score : ', )
    print("")
    print("-"*50)

--------------------------------------------------

Accuracy :  0.7864314317703247

--------------------------------------------------


In [13]:
TotalAccuracy = 0

with torch.no_grad():
    for num, data in enumerate(test_set):
        imgs, label = data
        imgs = imgs.to(device)
        label = label.to(device)
        
        prediction = net(imgs)
        
        correct_prediction = torch.argmax(prediction, 1) == label
        
        accuracy = correct_prediction.float().mean()
        TotalAccuracy = TotalAccuracy + accuracy
    TotalAccuracy = TotalAccuracy / num

    print(prediction)
    print(torch.argmax(prediction, 1))
    print(label)
    print(torch.argmax(prediction, 1) == label)

tensor([[30.1312,  4.9286],
        [ 2.3512, 15.9724],
        [ 2.1845,  5.6641],
        [ 5.5833, 13.2032],
        [ 2.4495, -2.6550],
        [10.2129, 19.4456],
        [ 6.9490,  3.7506]], device='cuda:0')
tensor([0, 1, 1, 1, 0, 1, 0], device='cuda:0')
tensor([1, 1, 1, 1, 1, 1, 1], device='cuda:0')
tensor([False,  True,  True,  True, False,  True, False], device='cuda:0')
